Roberta Modell

Wichtig: 
- filme_rezensionen_extendet.json muss im selben Verzeichnis liegen wie dieses Notebook
- die Modelle sind entsprechenden Personen zugeteilt. Zum ausführen jeweils # entfernen. Nach dem Durchlauf wird automatisch eine CSV mit den Sentiments gespeichert. Bitte die CSV dann mir zusenden bzw. in die Cloud laden und mir Bescheid geben.

Danke!!!

In [ ]:
import pandas as pd
import json



In [ ]:
file_path="filme_rezensionen_extendet.json"
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [ ]:
list_rows = [] #Liste für Zeilendaten

for film_id, film_info in data.items():          #Interation durch jedes Filmobjekt
    kategorie = ', '.join(film_info.get('kategorie', []))
    gesamtbewertung = film_info.get('gesamtbewertung', [])
    for rezension in film_info['rezensionen']: #für jede Rezension wird neues Dictionary erstellt
        
        row ={ 
            'film_id':film_id,
            'titel': film_info['titel'],
            'jahr': film_info['jahr'],
            'kategorie':kategorie,
            'review_id': rezension['data-review-id'],
            'username': rezension['username'],
            'gesamtbewertung': gesamtbewertung,
            'rating': rezension['rating'],
            'date': rezension['date'],
            'review_caption':rezension['title'],
            'bewertung':rezension['text']
        }
        list_rows.append(row)           # jedes Dictionary wird der Liste zugefügt

StarWars = pd.DataFrame(list_rows)     # jedes Dictionary wird als Zeile dem Pandas DataFrame hinzugefügt

StarWars['ID'] = range(1, len(StarWars) + 1) # einen Index hinzufügen und an die erste Position setzen
StarWars['ID'] = StarWars['ID'].astype(str)
spalten = ['ID'] + [spalte for spalte in StarWars.columns if spalte != 'ID']
StarWars = StarWars[spalten]
StarWars.head()
        

In [ ]:
StarWars['date'] = pd.to_datetime(StarWars['date'], format='%d %B %Y') # Datum umwandeln
StarWars['rating'] = pd.to_numeric(StarWars['rating'], errors='coerce') #in numerische Daten umwandeln
StarWars['gesamtbewertung'] = pd.to_numeric(StarWars['gesamtbewertung'], errors='coerce')

StarWars['titel'] =StarWars['titel'].replace('Das Imperium schlägt zurück', 'Star Wars: Episode V - Das Imperium schlägt zurück')
StarWars['titel'] =StarWars['titel'].replace('Krieg der Sterne', 'Star Wars: Episode IV - Krieg der Sterne')
StarWars['titel'] =StarWars['titel'].replace('Die Rückkehr der Jedi-Ritter', 'Star Wars: Episode VI - Die Rückkehr der Jedi-Ritter')
StarWars['titel'] =StarWars['titel'].replace('Kampf um Endor', 'Star Wars: Ewoks - Kampf um Endor')
StarWars['titel'] =StarWars['titel'].replace('Star Wars - Episode VII: Das Erwachen der Macht', 'Star Wars: Episode VII - Das Erwachen der Macht')
StarWars['titel'] =StarWars['titel'].replace('Star Wars - Episode IX: Der Aufstieg Skywalkers', 'Star Wars: Episode IX - Der Aufstieg Skywalkers')
StarWars['titel'] =StarWars['titel'].replace('Star Wars - Episode VIII: Die letzten Jedi', 'Star Wars: Episode VIII - Die letzten Jedi')

StarWars['rating_na'] = StarWars['rating'].isna() #einfügen einer Spalte, die TRUE für NA Werte in 'Rating' setzt
NaN_Filme_Rating = StarWars.groupby(['film_id', 'titel'])['rating_na'].sum() #Anzahl NA Werte gruppiert nach Film

StarWars.dropna(subset=['rating'], inplace=True)

NaN_Filme_Gesamtbewertung = StarWars[StarWars['gesamtbewertung'].isna()]

film = 'Star Wars: Episode III - Die Rache der Sith' # Kategorie und Gesamtbewertung bei Episode 3 fehlt -> hinzufügen

kategorie_add = 'Action, Abenteuer, Fantasy'
gesamtbewertung_add = 7.6
StarWars.loc[StarWars['titel'] == film, 'kategorie'] = kategorie_add
StarWars.loc[StarWars['titel'] == film, 'gesamtbewertung'] = gesamtbewertung_add

StarWars['film_id'] = StarWars['film_id'].replace({'tt1185834': 'tt0458290'})

titel_entfernen = ['Star Wars: Larry', 'Star Wars: Die Abenteuer der jungen Jedi']
StarWars = StarWars[~StarWars['titel'].isin(titel_entfernen)]

del StarWars['rating_na']

In [ ]:
#ggf. müssen bestimmte Pakete zunächst installiert werden

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm import tqdm
tqdm.pandas()

In [ ]:
# Laden des vortrainierten Sentiment-Modells
sentiment_model = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(sentiment_model)
model = AutoModelForSequenceClassification.from_pretrained(sentiment_model)

In [ ]:
StarWars_Roberta=StarWars[['ID', 'titel','gesamtbewertung','rating','bewertung']].copy()

In [ ]:
def sentiment_Roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt', padding=True,  truncation=True, max_length=512)
    output = model(**encoded_text)
    scores = output.logits[0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg': scores[0],
        'roberta_neu': scores[1],
        'roberta_pos': scores[2]
    }
    return scores_dict

In [ ]:
# Datensatz splitten in viertel

viertel=len(StarWars_Roberta)//4

StarWars_Roberta_1 = StarWars_Roberta.iloc[:viertel]
StarWars_Roberta_2 = StarWars_Roberta.iloc[viertel:2*viertel]
StarWars_Roberta_3 = StarWars_Roberta.iloc[2*viertel:3*viertel]
StarWars_Roberta_4 = StarWars_Roberta.iloc[3*viertel:]

Ivo: Roberta_2 (Hastags entsprechend entfernen zum laden)

In [ ]:


#StarWars_Roberta_2[['roberta_neg','roberta_neu','roberta_pos']] = StarWars_Roberta_2['bewertung'].progress_apply(lambda x: pd.Series(sentiment_Roberta(x)))
#StarWars_Roberta_2.to_csv('StarWars_Roberta_2.csv', index=False)

Miki: Roberta_3 (Hastags entsprechend entfernen zum laden)

In [ ]:
#StarWars_Roberta_3[['roberta_neg','roberta_neu','roberta_pos']] = StarWars_Roberta_3['bewertung'].progress_apply(lambda x: pd.Series(sentiment_Roberta(x)))
#StarWars_Roberta_3.to_csv('StarWars_Roberta_3.csv', index=False)

Arne: Roberta_4 (Hastags entsprechend entfernen zum laden)

In [ ]:
#StarWars_Roberta_4[['roberta_neg','roberta_neu','roberta_pos']] = StarWars_Roberta_4['bewertung'].progress_apply(lambda x: pd.Series(sentiment_Roberta(x)))
#StarWars_Roberta_4.to_csv('StarWars_Roberta_4.csv', index=False)